In [ ]:
import numpy as np

class Grid: # Environment
  def __init__(self, width, height, start):
    # i is vertical axis, j is horizontal
    self.width = width
    self.height = height
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions, obey_prob):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions
    self.obey_prob = obey_prob

  def non_terminal_states(self):
    return self.actions.keys()

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, s):
    return s not in self.actions

  def stochastic_move(self, action):
    p = np.random.random()
    if p <= self.obey_prob:
      return action
    if action == 'U' or action == 'D':
      return np.random.choice(['L', 'R'])
    elif action == 'L' or action == 'R':
      return np.random.choice(['U', 'D'])

  def move(self, action):
    actual_action = self.stochastic_move(action)
    if actual_action in self.actions[(self.i, self.j)]:
      if actual_action == 'U':
        self.i -= 1
      elif actual_action == 'D':
        self.i += 1
      elif actual_action == 'R':
        self.j += 1
      elif actual_action == 'L':
        self.j -= 1
    return self.rewards.get((self.i, self.j), 0)

  def check_move(self, action):
    i = self.i
    j = self.j
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      if action == 'U':
        i -= 1
      elif action == 'D':
        i += 1
      elif action == 'R':
        j += 1
      elif action == 'L':
        j -= 1
    # return a reward (if any)
    reward = self.rewards.get((i, j), 0)
    return ((i, j), reward)

  def get_transition_probs(self, action):
    # returns a list of (probability, reward, s') transition tuples
    probs = []
    state, reward = self.check_move(action)
    probs.append((self.obey_prob, reward, state))
    disobey_prob = 1 - self.obey_prob
    if not (disobey_prob > 0.0):
      return probs
    if action == 'U' or action == 'D':
      state, reward = self.check_move('L')
      probs.append((disobey_prob / 2, reward, state))
      state, reward = self.check_move('R')
      probs.append((disobey_prob / 2, reward, state))
    elif action == 'L' or action == 'R':
      state, reward = self.check_move('U')
      probs.append((disobey_prob / 2, reward, state))
      state, reward = self.check_move('D')
      probs.append((disobey_prob / 2, reward, state))
    return probs

  def game_over(self):

    return (self.i, self.j) not in self.actions

  def all_states(self):
    # possibly buggy but simple way to get all states
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())


def standard_grid(obey_prob=1.0, step_cost=None):
  g = Grid(3, 4, (2, 0))
  rewards = {(0, 3): 1, (1, 3): -1}
  actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('L', 'R'),
    (0, 2): ('L', 'D', 'R'),
    (1, 0): ('U', 'D'),
    (1, 2): ('U', 'D', 'R'),
    (2, 0): ('U', 'R'),
    (2, 1): ('L', 'R'),
    (2, 2): ('L', 'R', 'U'),
    (2, 3): ('L', 'U'),
  }
  g.set(rewards, actions, obey_prob)
  if step_cost is not None:
    g.rewards.update({
      (0, 0): step_cost,
      (0, 1): step_cost,
      (0, 2): step_cost,
      (1, 0): step_cost,
      (1, 2): step_cost,
      (2, 0): step_cost,
      (2, 1): step_cost,
      (2, 2): step_cost,
      (2, 3): step_cost,
    })
  return g

In [ ]:
# now just for check make object of Grid
grid = standard_grid(0.8,-0.4)
print(grid.all_states())
print(grid.non_terminal_states())

{(0, 0), (1, 3), (2, 1), (2, 3), (1, 0), (0, 3), (0, 1), (1, 2), (2, 0), (2, 2), (0, 2)}
dict_keys([(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2), (2, 3)])


In [79]:
SMALL_ENOUGH = 1e-3
GAMMA = 1
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

def print_values(V, g):
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")

def best_action_value(grid, V, s):
  # finds the highest value action (max_a) from state s, returns the action and value
  best_a = None
  best_value = float('-inf')
  grid.set_state(s)
  # loop through all possible actions to find the best current action
  for a in ALL_POSSIBLE_ACTIONS:
    transititions = grid.get_transition_probs(a)
    expected_v = 0
    expected_r = 0
    for (prob, r, state_prime) in transititions:
      expected_r += prob * r
      expected_v += prob * V[state_prime]
    v = expected_r + GAMMA * expected_v
    if v > best_value:
      best_value = v
      best_a = a
  return best_a, best_value

def calculate_values(grid):
  # initialize V(s)
  V = {}
  states = grid.all_states()
  for s in states:
    V[s] = 0

  iteration = 0
  while (iteration<=100000):
    # biggest_change is referred to by the mathematical symbol delta in equations
    biggest_change = 0
    state_with_action = grid.non_terminal_states()
    # print("Inside problem")
    # print(state_with_action)
    # count = 0
    for s in state_with_action:
      # previous = biggest_change
      old_v = V[s]
      _, new_v = best_action_value(grid, V, s)
      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - new_v))
    iteration+= 1
    if (biggest_change < SMALL_ENOUGH):
      print("Biggest Change", biggest_change)
      break
  return V


In [ ]:
grid = standard_grid(obey_prob=0.8, step_cost=-0.04)
# print rewards
print("rewards:")
print_values(grid.rewards,grid)

# calculate accurate values for each square
V = calculate_values(grid)

# print values
print("values regarding reward for each state remaining goal state is -0.04:")
print_values(V, grid)

rewards:
---------------------------
-0.04|-0.04|-0.04| 1.00|
---------------------------
-0.04| 0.00|-0.04|-1.00|
---------------------------
-0.04|-0.04|-0.04|-0.04|
values regarding reward for each state remaining goal state is -0.04:
---------------------------
 0.85| 0.91| 0.96| 0.00|
---------------------------
 0.80| 0.00| 0.70| 0.00|
---------------------------
 0.75| 0.70| 0.65| 0.43|


In [ ]:
grid = standard_grid(obey_prob=0.8, step_cost=-2)
# print rewards
print("rewards:")
print_values(grid.rewards,grid)

# calculate accurate values for each square
V = calculate_values(grid)

# print values
print("values regarding reward for each state remaining goal state is -2:")
print_values(V, grid)

rewards:
---------------------------
-2.00|-2.00|-2.00| 1.00|
---------------------------
-2.00| 0.00|-2.00|-1.00|
---------------------------
-2.00|-2.00|-2.00|-2.00|
values regarding reward for each state remaining goal state is -2:
---------------------------
-5.04|-2.23| 0.27| 0.00|
---------------------------
-7.54| 0.00|-1.57| 0.00|
---------------------------
-8.82|-6.47|-3.97|-1.77|


In [76]:
grid = standard_grid(obey_prob=0.8, step_cost=0.1)
# print rewards
print("rewards:")
print_values(grid.rewards,grid)

# calculate accurate values for each square
V = calculate_values(grid)

# print values
print("values regarding reward for each state remaining goal state is 0.1:")
print_values(V, grid)

rewards:
---------------------------
 0.10| 0.10| 0.10| 1.00|
---------------------------
 0.10| 0.00| 0.10|-1.00|
---------------------------
 0.10| 0.10| 0.10| 0.10|
values regarding reward for each state remaining goal state is 0.1:
---------------------------
 1735.27| 1735.36| 1735.41| 0.00|
---------------------------
 1735.36| 0.00| 1735.22| 0.00|
---------------------------
 1735.45| 1735.53| 1735.59| 1735.03|


In [80]:
grid = standard_grid(obey_prob=0.8, step_cost=0.02)
# print rewards
print("rewards:")
print_values(grid.rewards,grid)

# calculate accurate values for each square
V = calculate_values(grid)

# print values
print("values regarding reward for each state remaining goal state is 0.02:")
print_values(V, grid)

rewards:
---------------------------
 0.02| 0.02| 0.02| 1.00|
---------------------------
 0.02| 0.00| 0.02|-1.00|
---------------------------
 0.02| 0.02| 0.02| 0.02|
values regarding reward for each state remaining goal state is 0.02:
---------------------------
 3470.23| 3470.25| 3470.26| 0.00|
---------------------------
 3470.25| 0.00| 3470.22| 0.00|
---------------------------
 3470.27| 3470.29| 3470.30| 3470.18|


In [81]:
grid = standard_grid(obey_prob=0.8, step_cost=1)
# print rewards
print("rewards:")
print_values(grid.rewards,grid)

# calculate accurate values for each square
V = calculate_values(grid)

# print values
print("values regarding reward for each state remaining goal state is 1:")
print_values(V, grid)

rewards:
---------------------------
 1.00| 1.00| 1.00| 1.00|
---------------------------
 1.00| 0.00| 1.00|-1.00|
---------------------------
 1.00| 1.00| 1.00| 1.00|
values regarding reward for each state remaining goal state is 1:
---------------------------
 173470.37| 173471.19| 173471.76| 0.00|
---------------------------
 173471.19| 0.00| 173469.82| 0.00|
---------------------------
 173472.11| 173472.92| 173473.50| 173467.89|
